<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Add-price-&amp;-category" data-toc-modified-id="Add-price-&amp;-category-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Add price &amp; category</a></span></li><li><span><a href="#Filter-uncommon-categories" data-toc-modified-id="Filter-uncommon-categories-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Filter uncommon categories</a></span></li><li><span><a href="#Aggregate-sales-and-price-per-week" data-toc-modified-id="Aggregate-sales-and-price-per-week-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Aggregate sales and price per week</a></span></li><li><span><a href="#filter-with-minimum-sales-timestamp" data-toc-modified-id="filter-with-minimum-sales-timestamp-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>filter with minimum sales timestamp</a></span></li><li><span><a href="#Interpolate-periodical-features" data-toc-modified-id="Interpolate-periodical-features-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Interpolate periodical features</a></span></li><li><span><a href="#Add-price-change-per-week" data-toc-modified-id="Add-price-change-per-week-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Add price change per week</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from tqdm import tqdm

import datetime

In [7]:
items_df = pd.read_csv('../data/olist_order_items_dataset.csv')
payments_df = pd.read_csv('../data/olist_order_payments_dataset.csv')
orders_df = pd.read_csv('../data/olist_orders_dataset.csv')
customers_df = pd.read_csv('../data/olist_customers_dataset.csv')
products_df = pd.read_csv('../data/olist_products_dataset.csv')
name_translation_df = pd.read_csv('../data/product_category_name_translation.csv', index_col=0)

### Add price & category

In [3]:
data = orders_df[['order_id', 'order_purchase_timestamp', 'order_status']]
data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp'])
data

/tmp/ipykernel_13831/530159787.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp'])


,order_id,order_purchase_timestamp,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,delivered
...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,2017-03-09 09:54:05,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,2018-02-06 12:58:58,delivered
99438,83c1379a015df1e13d02aae0204711ab,2017-08-27 14:46:43,delivered
99439,11c177c8e97725db2631073c19f07b62,2018-01-08 21:28:27,delivered


In [4]:
data = data.merge(items_df[['order_id', 'product_id', 'price', 'freight_value']], on='order_id')
data = data.merge(products_df[['product_id', 'product_category_name','product_weight_g','product_length_cm', 'product_height_cm', 'product_width_cm']], on='product_id')
data

,order_id,order_purchase_timestamp,order_status,product_id,price,freight_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,delivered,87285b34884572647811a353c7ac498a,29.99,8.72,utilidades_domesticas,500.0,19.0,8.0,13.0
1,128e10d95713541c87cd1a2e48201934,2017-08-15 18:29:31,delivered,87285b34884572647811a353c7ac498a,29.99,7.78,utilidades_domesticas,500.0,19.0,8.0,13.0
2,0e7e841ddf8f8f2de2bad69267ecfbcf,2017-08-02 18:24:47,delivered,87285b34884572647811a353c7ac498a,29.99,7.78,utilidades_domesticas,500.0,19.0,8.0,13.0
3,bfc39df4f36c3693ff3b63fcbea9e90a,2017-10-23 23:26:46,delivered,87285b34884572647811a353c7ac498a,29.99,14.10,utilidades_domesticas,500.0,19.0,8.0,13.0
4,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,delivered,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76,perfumaria,400.0,19.0,13.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...
112645,e8fd20068b9f7e6ec07068bb7537f781,2017-08-10 21:21:07,delivered,0df37da38a30a713453b03053d60d3f7,356.00,18.12,esporte_lazer,2800.0,28.0,23.0,23.0
112646,e8fd20068b9f7e6ec07068bb7537f781,2017-08-10 21:21:07,delivered,0df37da38a30a713453b03053d60d3f7,356.00,18.12,esporte_lazer,2800.0,28.0,23.0,23.0
112647,cfa78b997e329a5295b4ee6972c02979,2017-12-20 09:52:41,delivered,3d2c44374ee42b3003a470f3e937a2ea,55.90,15.14,instrumentos_musicais,300.0,16.0,6.0,12.0
112648,9c5dedf39a927c1b2549525ed64a053c,2017-03-09 09:54:05,delivered,ac35486adb7b02598c182c2ff2e05254,72.00,13.08,beleza_saude,1175.0,22.0,13.0,18.0


### Filter uncommon categories

In [5]:
top_category = data.product_category_name.value_counts().iloc[:50].index.to_list()
top_category

['cama_mesa_banho',
 'beleza_saude',
 'esporte_lazer',
 'moveis_decoracao',
 'informatica_acessorios',
 'utilidades_domesticas',
 'relogios_presentes',
 'telefonia',
 'ferramentas_jardim',
 'automotivo',
 'brinquedos',
 'cool_stuff',
 'perfumaria',
 'bebes',
 'eletronicos',
 'papelaria',
 'fashion_bolsas_e_acessorios',
 'pet_shop',
 'moveis_escritorio',
 'consoles_games',
 'malas_acessorios',
 'construcao_ferramentas_construcao',
 'eletrodomesticos',
 'instrumentos_musicais',
 'eletroportateis',
 'casa_construcao',
 'livros_interesse_geral',
 'alimentos',
 'moveis_sala',
 'casa_conforto',
 'bebidas',
 'audio',
 'market_place',
 'construcao_ferramentas_iluminacao',
 'climatizacao',
 'moveis_cozinha_area_de_servico_jantar_e_jardim',
 'alimentos_bebidas',
 'industria_comercio_e_negocios',
 'livros_tecnicos',
 'telefonia_fixa',
 'fashion_calcados',
 'construcao_ferramentas_jardim',
 'eletrodomesticos_2',
 'agro_industria_e_comercio',
 'artes',
 'pcs',
 'sinalizacao_e_seguranca',
 'construc

In [6]:
uncommon = data['product_category_name'].apply(lambda x: False if x in top_category else True)
data = data[~uncommon]

### Aggregate sales and price per week

In [7]:
data['sales'] = 1
data_sales = data.groupby(['product_category_name', pd.Grouper(key='order_purchase_timestamp', freq='d')])['sales'] \
       .sum() \
       .reset_index() \
       .sort_values('order_purchase_timestamp')
data_sales

/tmp/ipykernel_13831/325356375.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sales'] = 1


,product_category_name,order_purchase_timestamp,sales
12502,moveis_decoracao,2016-09-04,2
16261,telefonia,2016-09-05,1
2408,beleza_saude,2016-09-15,3
1647,bebes,2016-10-02,1
15575,relogios_presentes,2016-10-03,1
...,...,...,...
3009,beleza_saude,2018-08-29,7
12500,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-08-29,2
10806,informatica_acessorios,2018-08-29,1
16145,relogios_presentes,2018-08-29,2


In [8]:
data_price = data.groupby(['product_category_name', pd.Grouper(key='order_purchase_timestamp', freq='d')])['price'] \
       .mean() \
       .reset_index() \
       .sort_values('order_purchase_timestamp')
data_price

,product_category_name,order_purchase_timestamp,price
12502,moveis_decoracao,2016-09-04,36.445000
16261,telefonia,2016-09-05,59.500000
2408,beleza_saude,2016-09-15,44.990000
1647,bebes,2016-10-02,100.000000
15575,relogios_presentes,2016-10-03,29.900000
...,...,...,...
3009,beleza_saude,2018-08-29,81.235714
12500,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-08-29,209.000000
10806,informatica_acessorios,2018-08-29,84.990000
16145,relogios_presentes,2018-08-29,134.400000


In [9]:
data = data_sales.merge(data_price, on=['product_category_name', 'order_purchase_timestamp'])
data

,product_category_name,order_purchase_timestamp,sales,price
0,moveis_decoracao,2016-09-04,2,36.445000
1,telefonia,2016-09-05,1,59.500000
2,beleza_saude,2016-09-15,3,44.990000
3,bebes,2016-10-02,1,100.000000
4,relogios_presentes,2016-10-03,1,29.900000
...,...,...,...,...
17610,beleza_saude,2018-08-29,7,81.235714
17611,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-08-29,2,209.000000
17612,informatica_acessorios,2018-08-29,1,84.990000
17613,relogios_presentes,2018-08-29,2,134.400000


### filter with minimum sales timestamp

In [10]:

n_price_point = data.groupby('product_category_name').order_purchase_timestamp.count()
product_cats = n_price_point[n_price_point > 8].index
data = data.set_index('product_category_name').loc[product_cats].reset_index()

print('total product_category_name left:', product_cats.shape[0])
print('rotal row left:', data.shape[0])

total product_category_name left: 50
rotal row left: 17615


In [11]:
data

,product_category_name,order_purchase_timestamp,sales,price
0,agro_industria_e_comercio,2017-01-23,2,21.990000
1,agro_industria_e_comercio,2017-01-31,1,21.990000
2,agro_industria_e_comercio,2017-02-05,1,21.990000
3,agro_industria_e_comercio,2017-02-08,1,21.990000
4,agro_industria_e_comercio,2017-02-12,1,92.900000
...,...,...,...,...
17610,utilidades_domesticas,2018-08-24,18,57.623889
17611,utilidades_domesticas,2018-08-25,5,139.938000
17612,utilidades_domesticas,2018-08-26,9,80.611111
17613,utilidades_domesticas,2018-08-27,13,77.245385


### Interpolate periodical features

In [12]:
def get_interpolated_timestamp(timestamp):
    """ Fill in missing mondays between timestamp gap
    
    NOTE: the timestamp series must already be sorted!
    """
    
    return pd.date_range(start=timestamp.iloc[0], end=timestamp.iloc[-1], freq='W-MON').to_frame(name='order_purchase_timestamp')

def apply_interpolation(product_sales_history, product_category_name):
    """ Fill in the gaps between sales & price timestamps """
    
    product_sales_history = product_sales_history.sort_values('order_purchase_timestamp')
    timestamp = get_interpolated_timestamp(product_sales_history['order_purchase_timestamp'])
    product_sales_history = product_sales_history.merge(timestamp, 
                                                        on='order_purchase_timestamp', 
                                                        how='outer').sort_values('order_purchase_timestamp')
    product_sales_history['sales'] = product_sales_history['sales'].fillna(0)
    product_sales_history['price'] = product_sales_history['price'].fillna(method='ffill')    
    product_sales_history['product_category_name'] = product_category_name
    
    return product_sales_history

In [13]:
from tqdm import tqdm

interpolated_data = []
for product_category_name, product in tqdm(data.groupby('product_category_name')):
    interpolated_data.append(apply_interpolation(product, product_category_name))
    

100%|███████████████████████████████████████████████████| 50/50 [00:00<00:00, 65.34it/s]


In [15]:
a = pd.concat(interpolated_data, axis=0)
a

,product_category_name,order_purchase_timestamp,sales,price
0,agro_industria_e_comercio,2017-01-23,2.0,21.990000
144,agro_industria_e_comercio,2017-01-30,0.0,21.990000
1,agro_industria_e_comercio,2017-01-31,1.0,21.990000
2,agro_industria_e_comercio,2017-02-05,1.0,21.990000
145,agro_industria_e_comercio,2017-02-06,0.0,21.990000
...,...,...,...,...
582,utilidades_domesticas,2018-08-24,18.0,57.623889
583,utilidades_domesticas,2018-08-25,5.0,139.938000
584,utilidades_domesticas,2018-08-26,9.0,80.611111
585,utilidades_domesticas,2018-08-27,13.0,77.245385


In [15]:
data = pd.concat(interpolated_data, axis=0)
data

,product_category_name,order_purchase_timestamp,sales,price
0,agro_industria_e_comercio,2017-01-23,2.0,21.990000
144,agro_industria_e_comercio,2017-01-30,0.0,21.990000
1,agro_industria_e_comercio,2017-01-31,1.0,21.990000
2,agro_industria_e_comercio,2017-02-05,1.0,21.990000
145,agro_industria_e_comercio,2017-02-06,0.0,21.990000
...,...,...,...,...
582,utilidades_domesticas,2018-08-24,18.0,57.623889
583,utilidades_domesticas,2018-08-25,5.0,139.938000
584,utilidades_domesticas,2018-08-26,9.0,80.611111
585,utilidades_domesticas,2018-08-27,13.0,77.245385


In [16]:
print('total product id left:', product_cats.shape[0])
print('rotal row left:', data.shape[0])

total product id left: 50
rotal row left: 19566


In [17]:
data.groupby('product_category_name').order_purchase_timestamp.count().mean()

391.32

### Add price change per week

In [22]:
data.shift(1)

,product_category_name,order_purchase_timestamp,sales,price
0,NaN,NaT,NaN,NaN
144,agro_industria_e_comercio,2017-01-23,2.0,21.990000
1,agro_industria_e_comercio,2017-01-30,0.0,21.990000
2,agro_industria_e_comercio,2017-01-31,1.0,21.990000
145,agro_industria_e_comercio,2017-02-05,1.0,21.990000
...,...,...,...,...
582,utilidades_domesticas,2018-08-23,7.0,82.028571
583,utilidades_domesticas,2018-08-24,18.0,57.623889
584,utilidades_domesticas,2018-08-25,5.0,139.938000
585,utilidades_domesticas,2018-08-26,9.0,80.611111


In [23]:
data

,product_category_name,order_purchase_timestamp,sales,price
0,agro_industria_e_comercio,2017-01-23,2.0,21.990000
144,agro_industria_e_comercio,2017-01-30,0.0,21.990000
1,agro_industria_e_comercio,2017-01-31,1.0,21.990000
2,agro_industria_e_comercio,2017-02-05,1.0,21.990000
145,agro_industria_e_comercio,2017-02-06,0.0,21.990000
...,...,...,...,...
582,utilidades_domesticas,2018-08-24,18.0,57.623889
583,utilidades_domesticas,2018-08-25,5.0,139.938000
584,utilidades_domesticas,2018-08-26,9.0,80.611111
585,utilidades_domesticas,2018-08-27,13.0,77.245385


In [24]:
data['price_diff'] = data['price'] - data['price'].shift(1)

In [25]:
data['price_discount'] = data['price_diff'].shift(-1) / data['price'] * -1

In [2]:
data['price'] = data['price'] * 3053.52
data

,product_category_name,order_purchase_timestamp,sales,price
0,moveis_decoracao,2016-09-04,2,111285.5364
1,telefonia,2016-09-05,1,181684.4400
2,moveis_decoracao,2016-09-05,1,122110.2648
3,moveis_decoracao,2016-09-06,1,122110.2648
4,moveis_decoracao,2016-09-07,1,122110.2648
...,...,...,...,...
38228,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-08-30,1,442760.4000
38229,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-08-31,1,442760.4000
38230,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-09-01,1,442760.4000
38231,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-09-02,1,442760.4000


In [3]:
data.to_csv('../data/prepared/interpolated_demand_history_by_category.csv', index=False)

In [4]:
data

,product_category_name,order_purchase_timestamp,sales,price
0,moveis_decoracao,2016-09-04,2,111285.5364
1,telefonia,2016-09-05,1,181684.4400
2,moveis_decoracao,2016-09-05,1,122110.2648
3,moveis_decoracao,2016-09-06,1,122110.2648
4,moveis_decoracao,2016-09-07,1,122110.2648
...,...,...,...,...
38228,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-08-30,1,442760.4000
38229,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-08-31,1,442760.4000
38230,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-09-01,1,442760.4000
38231,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-09-02,1,442760.4000


In [25]:
data = pd.read_csv('../data/prepared/interpolated_demand_history_by_category.csv')

In [26]:
data

,product_category_name,order_purchase_timestamp,sales,price
0,moveis_decoracao,2016-09-04,2,111285.5364
1,telefonia,2016-09-05,1,181684.4400
2,moveis_decoracao,2016-09-05,1,122110.2648
3,moveis_decoracao,2016-09-06,1,122110.2648
4,moveis_decoracao,2016-09-07,1,122110.2648
...,...,...,...,...
37802,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-08-30,1,442760.4000
37803,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-08-31,1,442760.4000
37804,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-09-01,1,442760.4000
37805,moveis_cozinha_area_de_servico_jantar_e_jardim,2018-09-02,1,442760.4000


In [8]:
name_translation_dict = name_translation_df['product_category_name_english'].to_dict()
name_translation_dict

{'beleza_saude': 'health_beauty',
 'informatica_acessorios': 'computers_accessories',
 'automotivo': 'auto',
 'cama_mesa_banho': 'bed_bath_table',
 'moveis_decoracao': 'furniture_decor',
 'esporte_lazer': 'sports_leisure',
 'perfumaria': 'perfumery',
 'utilidades_domesticas': 'housewares',
 'telefonia': 'telephony',
 'relogios_presentes': 'watches_gifts',
 'alimentos_bebidas': 'food_drink',
 'bebes': 'baby',
 'papelaria': 'stationery',
 'tablets_impressao_imagem': 'tablets_printing_image',
 'brinquedos': 'toys',
 'telefonia_fixa': 'fixed_telephony',
 'ferramentas_jardim': 'garden_tools',
 'fashion_bolsas_e_acessorios': 'fashion_bags_accessories',
 'eletroportateis': 'small_appliances',
 'consoles_games': 'consoles_games',
 'audio': 'audio',
 'fashion_calcados': 'fashion_shoes',
 'cool_stuff': 'cool_stuff',
 'malas_acessorios': 'luggage_accessories',
 'climatizacao': 'air_conditioning',
 'construcao_ferramentas_construcao': 'construction_tools_construction',
 'moveis_cozinha_area_de_ser

In [ ]:
data['productCategory'] = data['productCategory'].apply(lambda x: name_translation_dict[x] if x in name_translation_dict.keys() else None)